<h1 style="text-align: center; font-weight: bold;">Data Sciense Course - Spring 1403</h1>
<h1 style="text-align: center; font-weight: bold;">Project Phase1 - Introduction to Data Science </h1>
<h1 style="text-align: center;">Mohammadreza Mohammadhashemi : 810100206</h1>
<h1 style="text-align: center;">Soheil Hajian Manesh : 810100119</h1>
<h1 style="text-align: center;">Mahdi Ebrahimi Soltani : 810100241</h1>


# Problem Description
in this phase we are going to scrapping datas from fiverr.com website using BeautifulSoup and selenium packages and then extract some meaningful statistics from dataset using EDA and visualisation and preprocessing steps.

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import os
from fiverr_api import session

## Data Collection

In [3]:
def initialize_driver(proxy=None):
    options = Options()
    options.add_argument(r"user-data-dir=C:\Users\Lenovo\AppData\Local\Google\Chrome\User Data")
    options.add_argument(r"profile-directory=Profile 1")
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
    options.add_argument(f"user-agent={user_agent}")
    driver = webdriver.Chrome(options=options)
    return driver

base_url = "https://www.fiverr.com"

In [4]:
def get_text_or_default(element, default="N/A"):
    return element.text.strip() if element else default

In [5]:
def extract_price_info(table):
    prices = table.find_all("div", class_="price-wrapper")
    if len(prices) == 0:
        return (
            "N/A",
            "N/A",
            "N/A",
        )
    return (
        get_text_or_default(prices[0]),
        get_text_or_default(prices[1]),
        get_text_or_default(prices[2]),
    )

In [6]:
def extract_price_info_single_plan(soup):
    price = soup.find("div", class_="price-wrapper")
    if price is None:
        return(
            "N/A",
            "N/A",
            "N/A",
        )
    return (
        get_text_or_default(price),
        get_text_or_default(price),
        get_text_or_default(price)
    )

In [7]:
def extract_delivery_info(table):
    row = table.find("tr", class_="delivery-time")
    if row is None:
        return(
            "N/A",
            "N/A",
            "N/A",
        )
    tds = row.find_all("td")
    delivery_0 = tds[1].find_all("label")[0] if len(tds[1].find_all("label"))>0 else tds[1]
    delivery_1 = tds[2].find_all("label")[0] if len(tds[2].find_all("label"))>0 else tds[2]
    delivery_2 = tds[3].find_all("label")[0] if len(tds[3].find_all("label"))>0 else tds[3]
    return (
            get_text_or_default(delivery_0),
            get_text_or_default(delivery_1),
            get_text_or_default(delivery_2),
        )

In [8]:
def extract_delivery_info_single_plan(soup):
    delivery = soup.find_all("div", class_="delivery-wrapper")[0]
    if delivery is None: 
        return (
            "N/A",
            "N/A",
            "N/A",
        )
    return (
        get_text_or_default(delivery),
        get_text_or_default(delivery),
        get_text_or_default(delivery),
    )

In [9]:
def extract_revision_info(table):
    rows = table.find_all("tr")
    revision_row = None
    for row in rows[1:]:
        if row.find_all("td")[0].text.strip() == "Revisions":
            revision_row = row
            break
    if  revision_row is None:
        return (
            "-1",
            "-1",
            "-1",
        )

    revisions = revision_row.find_all("td") 
    return (
        get_text_or_default(revisions[1]),
        get_text_or_default(revisions[2]),
        get_text_or_default(revisions[3]),
    )

In [10]:
def extract_revision_info_single_plan(soup):
    revision_all = soup.find_all("div", class_="revisions-wrapper")
    if len(revision_all) == 0:   
        return (
            "-1",
            "-1",
            "-1",
        )
    revision = revision_all[0]
    return(
        get_text_or_default(revision),
        get_text_or_default(revision),
        get_text_or_default(revision),
    )

In [11]:
def extract_user_stats(soup):
    seller_card=soup.find("div",class_="seller-card")
    user_stats = seller_card.find("ul", class_="user-stats")
    country = "N/A"
    member_since = "N/A"
    avg_response_time = "N/A"
    last_delivery = "N/A"
    language = "N/A"
    if user_stats:
        items=user_stats.find_all("li")
        for item in items:
            if "From" in item.text.strip():
                country = get_text_or_default(item.find("strong"))
            elif "Member since" in item.text.strip():
                member_since = get_text_or_default(item.find("strong"))
            elif "Avg. response time" in item.text.strip():
                avg_response_time = get_text_or_default(item.find("strong"))
            elif "Last delivery" in item.text.strip():
                last_delivery = get_text_or_default(item.find("strong"))
            elif "Languages" in item.text.strip():
                language = get_text_or_default(item.find("strong"))

        return (
            country,
            member_since,
            avg_response_time,
            last_delivery,
            language,
        )
    else : 
        country = seller_card.find_all("p",class_="jpwn1n20u jpwn1n151 jpwn1n138 jpwn1n6 jpwn1n2")[0]
        language = seller_card.find_all("span",class_="jpwn1n20u jpwn1n151 jpwn1n138 jpwn1n6 jpwn1n2")[0]
        return(
            get_text_or_default(country),
            "N/A",
            "N/A",
            "N/A",
            get_text_or_default(language),
        )

In [12]:
def extract_rating_info(soup):
    rating = get_text_or_default(soup.find("b", class_="rating-score"))
    rating_count = get_text_or_default(soup.find("span", class_="rating-count-number"))
    return rating, rating_count

In [13]:
def extract_order_in_queue(soup):
    order_in_queue = soup.find("div", class_="FVVUrQM")
    return get_text_or_default(order_in_queue)

In [14]:
def extract_records(driver, url, category, field, level, sellers_in_same_level):
    try:
        driver.get(url)

        body = driver.find_element(By.CLASS_NAME, "layout-service")

        html_content = body.get_attribute("innerHTML")

        soup = BeautifulSoup(html_content, "html.parser")

        table = soup.find("div", class_="gig-page-packages-table")

        if table:
            is_single_plan = False
            basic_price, standard_price, premium_price = extract_price_info(table)
            basic_delivery, standard_delivery, premium_delivery = extract_delivery_info(table)
            basic_revision, standard_revision, premium_revision = extract_revision_info(table)
        else:
            is_single_plan = True
            basic_price, standard_price, premium_price = extract_price_info_single_plan(soup)
            basic_delivery, standard_delivery, premium_delivery = extract_delivery_info_single_plan(soup)
            basic_revision, standard_revision, premium_revision = extract_revision_info_single_plan(soup)
        rating, rating_count = extract_rating_info(soup)
            
        country, member_since, avg_response_time, last_delivery, language = extract_user_stats(soup)

        order_in_queue = extract_order_in_queue(soup)

        return (
                category, field, level, sellers_in_same_level,
                basic_price, standard_price, premium_price,
                basic_delivery, standard_delivery, premium_delivery,
                basic_revision, standard_revision, premium_revision,
                rating, rating_count, country, member_since,
                avg_response_time, last_delivery, language, order_in_queue,
                is_single_plan
            )
    except Exception as e:
        return None

In [15]:
columns = [
    "Category",
    "Field",
    "Seller Level",
    "Seller In Same Level",
    "Basic Price",
    "Standard Price",
    "Premium Price",
    "Basic Delivery",
    "Standard Delivery",
    "Premium Delivery",
    "Basic Revision",
    "Standard Revision",
    "Premium Revision",
    "Rating",
    "Rating Count",
    "Country",
    "Member Since",
    "Avg Response Time",
    "Last Delivery",
    "Language",
    "Order in Queue",
    "is_single_plan",
]

In [31]:
def process_html_files(base_path):
    error_links = []
    driver = initialize_driver()
    for category in os.listdir(base_path):
        category_path = os.path.join(base_path, category)
        if os.path.isdir(category_path):
            for field in os.listdir(category_path):
                field_path = os.path.join(category_path, field)
                if os.path.isdir(field_path):
                    for level in os.listdir(field_path):
                        records = []
                        level_path = os.path.join(field_path, level)
                        sellers_in_same_level = int(level.split("_")[-1])
                        for item in os.listdir(level_path):
                            item_path = os.path.join(level_path, item)
                            if os.path.isfile(item_path):
                                seller_level = level.split("_")[0]
                                print(item_path)
                                item_url = os.path.abspath(item_path)
                                record = extract_records(driver,item_url,category,field,seller_level,sellers_in_same_level,)
                                if record:
                                    records.append(record)
                                else:
                                    print("null return")
                                    error_links.append(item_path)
                        field_folder = os.path.join("CSVs", category, field)
                        os.makedirs(field_folder, exist_ok=True)
                        csv_file_path = os.path.join(
                            field_folder, f"{seller_level}.csv"
                        )
                        field_df = pd.DataFrame(records, columns=columns)
                        field_df.to_csv(csv_file_path, index=False)
    driver.quit()
    return error_links

In [32]:
def create_and_process_html_files(base_path):
    error_links = []
    driver = initialize_driver()
    session.set_scraper_api_key("046f60c9745770c0c800a295b5a3e56e")
    temp_file_path = 'temp.html'
    for category in os.listdir(base_path):
        category_path = os.path.join(base_path, category)
        if os.path.isdir(category_path):
            for field in os.listdir(category_path):
                field_path = os.path.join(category_path, field)
                if os.path.isdir(field_path):
                    for file_name in ["0.html","1.html","2.html", "3.html"]:
                        file_path = os.path.join(field_path, file_name)
                        if os.path.isfile(file_path):
                            records = []
                            with open(file_path, "r", encoding="utf-8") as file:
                                html_content = file.read()
                            soup = BeautifulSoup(html_content, "html.parser")
                            item_divs = soup.find_all("div", class_="gig-card-layout")
                            sellers_in_same_level = get_text_or_default(soup.find("p", class_="number-of-results"))
                            item_links = []
                            for div in item_divs:
                                link = div.find_all("a", href=True)[2]
                                if link:
                                    item_links.append("https://www.fiverr.com" + link["href"])
                            seller_level = ["new seller","level 1","level 2","top rated seller"][int(file_name[0])]
                            for link in item_links:
                                print(link)
                                response = session.get(link)
                                with open(temp_file_path, 'w', encoding='utf-8') as temp_file:
                                    temp_file.write(str(response.soup))
                                page_url = os.path.abspath(temp_file_path)
                                record = extract_records(driver, page_url, category, field, seller_level, sellers_in_same_level)
                                if record:
                                    records.append(record)
                                else:
                                    print("null return")
                                    error_links.append(link)

                            field_folder = os.path.join("CSVs", category, field)
                            os.makedirs(field_folder, exist_ok=True)
                            csv_file_path = os.path.join(field_folder, f"{seller_level}.csv")
                            field_df = pd.DataFrame(records, columns=columns)
                            field_df.to_csv(csv_file_path, index=False)

    driver.quit()
    return error_links

In [ ]:
base_path = "htmls"
errors = create_and_process_html_files(base_path)

In [ ]:
base_path = "items_htmls"
errors = process_html_files(base_path)

In [29]:
df_list = []

for category in os.listdir("CSVs"):
    category_path = os.path.join("CSVs", category)
    for field in os.listdir(category_path):
        field_path = os.path.join(category_path, field)
        if os.path.isdir(field_path):
            for csv_file in os.listdir(field_path):
                csv_path = os.path.join(field_path, csv_file)
                df = pd.read_csv(csv_path)
                df_list.append(df)
df=pd.read_csv("Business.csv")
df_list.append(df)

fiverr_df = pd.concat(df_list, ignore_index=True)

print(fiverr_df.shape)

(6262, 22)


In [30]:
fiverr_df.to_csv("Fiverr Dataset", index=False)